# Address Parser

Is able to recognize named entities and format outputs based on a defined schema

In [1]:
GCP_PROJECT_NAME = "single-azimuth-413609"

In [2]:
!pip install --quiet --upgrade google-cloud-aiplatform 
!pip install --quiet --upgrade google-auth



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
!gcloud auth application-default login --scopes=openid,https://www.googleapis.com/auth/userinfo.email,https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/sqlservice.login,https://www.googleapis.com/auth/drive

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=4oCvq3SgCR1jnwxtixGt6ZhLgBjEdr&access_type=offline&code_challenge=02T-ecmi9TvqMRQBMLwo_sVSitQqmN2SprRemd3fwfU&code_challenge_method=S256


Credentials saved to file: [/home/vincent/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "single-azimuth-413609" was added to ADC which can be used by Google client libraries for billing and quota. Note that 

In [4]:
!gcloud config set project {GCP_PROJECT_NAME}

Updated property [core/project].


In [5]:
!gcloud services enable aiplatform.googleapis.com

In [8]:
# Importing the library 
import google.auth

# Setting up credentials
credentials, _ = google.auth.default()
# authed_http = google.auth.transport.requests.AuthorizedSession(credentials)

In [9]:
import vertexai
from vertexai.preview.language_models import TextGenerationModel

vertexai.init(project=GCP_PROJECT_NAME, location="us-central1")
parameters = {
    "candidate_count": 1,
    "max_output_tokens": 126,
    "temperature": 0,
    "top_p": 1
}
model = TextGenerationModel.from_pretrained("text-bison")


## Basic french address

In [10]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: {
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 1, rue de la Paix, 75008 Paris
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "1",
  "street_type": "rue",
  "street_name": "de la Paix",
  "zip_code": "75008",
  "city": "Paris",
  "confidence": 1
}


### Unusual French address

In [11]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: {
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 78 bis Cavee des Ecameaux 76500 Elbeuf
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "78",
  "street_multiplier": "bis",
  "street_type": "Cavee",
  "street_name": "des Ecameaux",
  "zip_code": "76500",
  "city": "Elbeuf",
  "confidence": 1
}


In [12]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 78 bis Cavee des Ecameaux 76500 Elbeuf
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "78",
  "street_multiplier": "bis",
  "street_type": "Cavee",
  "street_name": "des Ecameaux",
  "zip_code": "76500",
  "city": "Elbeuf",
  "confidence": 1
}


## Complex US address

In [13]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: {
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: C/o John Doe LLC, 111, 8th Ave Ste 1509, Oklahoma, 74136-1922, USA
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "111",
  "street_multiplier": "8th",
  "street_type": "Ave",
  "street_name": "Ste 1509",
  "zip_code": "74136-1922",
  "city": "Oklahoma",
  "country": "USA",
  "confidence": 1
}


## German address 

generator at: https://tarjeta-credito.net/cvv/address-gen-germany/

In [14]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: {
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: Haßfurter Str. 1, 91056 Erlangen, Germany.
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "1",
  "street_type": "Str.",
  "street_name": "Haßfurter",
  "zip_code": "91056",
  "city": "Erlangen",
  "country": "Germany",
  "confidence": 1
}


### german address with Country not mentionned

In [15]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city and country
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: Text: 37, rue du champ du pardon, 76000 rouen
JSON: {
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: {
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}

Text: Auschnippe 9, 37170 Uslar
JSON:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  {
  "street_number": "9",
  "street_name": "Auschnippe",
  "zip_code": "37170",
  "city": "Uslar",
  "country": "Germany",
  "confidence": 1
}


## Confidence level w/ False address

In [16]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: 
{
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: 
{
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"confidence\": 1
}

Text: ici c'est paris !, 75001 Paris
JSON:

""",
    **parameters
)
print(f"{response.text}")

 {
  "street_number": null,
  "street_type": null,
  "street_name": null,
  "zip_code": "75001",
  "city": "Paris",
  "confidence": 0.5
}


### Confidence level w/ False address and Country not mentioned

In [17]:
response = model.predict(
    """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input

Text: 37, rue du champ du pardon, 76000 rouen
JSON: 
{
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}

Text: 10-82 Cavée St Gervais, 76000 rouen
JSON: 
{
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}

Text: 452 quater grande rue general 85000 le palais vendee
JSON:

""",
    **parameters
)
print(f"{response.text}")


 {
  "street_number": "452",
  "street_multiplier": "quater",
  "street_type": "grande rue",
  "street_name": "general",
  "zip_code": "85000",
  "city": "le palais",
  "department": "vendee",
  "country": "France",
  "confidence": 1
}


## Programmatic

Using Langchain we can have a more programmatic way 

In [18]:
!pip install --upgrade --quiet  langchain-core langchain-google-vertexai


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [19]:
from langchain_google_vertexai import VertexAI

# candidate count parameter missing ?
model = VertexAI(model_name="text-bison", top_p=1, temperature=0, max_output_tokens=126)

In [20]:

from langchain_core.pydantic_v1 import BaseModel
from typing import Optional

from langchain_core.output_parsers import JsonOutputParser


class Address(BaseModel):
    street_number: Optional[str]
    street_multiplier: Optional[str]
    street_type: Optional[str]
    street_name: Optional[str]
    zip_code: Optional[str]
    city: Optional[str]
    country: Optional[str]
    confidence: int


parser = JsonOutputParser(pydantic_object=Address)
# pprint(parser.get_format_instructions())


In [21]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate

examples = [
    {
        "address": "37, rue du champ du pardon, 76000 rouen",
        "answer": """{{
  \"street_number\": \"37\",
  \"street_type\": \"rue\",
  \"street_name\": \"du champ du pardon\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}}"""
    },
    {
        "address": "10-82 Cavée St Gervais, 76000 rouen",
        "answer": """{{
  \"street_number\": \"10-82\",
  \"street_type\": \"Cavée\",
  \"street_name\": \"St Gervais\",
  \"zip_code\": \"76000\",
  \"city\": \"rouen\",
  \"country\": \"France\",
  \"confidence\": 1
}}"""
    },
]

prefix = """Extract and split the address parameters from the text below in a JSON format.
be very precise, differentiate the street number, multiplier, street type, street name, zip code, city
provide a rating number to indicate your confidence level. 0 is the lowest confidence, 1 is the maximum.
provide only one response for one address input"""

example_prompt = PromptTemplate(
    template="""Text: {address}\nJSON: {answer}""",
    # template="""Answer the user query.\n{format_instructions}\nText: {address}\nJSON: {answer}\n""",
    input_variables=["address", "answer"],
    # partial_variables={"format_instructions": parser.get_format_instructions()}
)

# print(example_prompt.format(**examples[0]))

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix="Text: {address}\nJSON:",
    input_variables=["address"],

)

# print(prompt.format(address="78 bis Cavee des Ecameaux 76500 Elbeuf"))


chain = prompt | model | parser

### Testing

In [22]:

address = "1 bis rue de la Paix, 75008 Paris"
Address(**chain.invoke({"address": address}))


Address(street_number='1', street_multiplier='bis', street_type='rue', street_name='de la Paix', zip_code='75008', city='Paris', country='France', confidence=1)

In [23]:

address = "Auschnippe 9, 37170 Uslar"
Address(**chain.invoke({"address": address}))


Address(street_number='9', street_multiplier=None, street_type=None, street_name='Auschnippe', zip_code='37170', city='Uslar', country='Germany', confidence=1)

In [24]:

address = "C/o John Doe LLC, 111, 8th Ave Ste 1509, Oklahoma, 74136-1922, USA"
add = Address(**chain.invoke({"address": address}))
add


Address(street_number='111', street_multiplier='8th', street_type='Ave', street_name='Ste 1509', zip_code='74136-1922', city='Oklahoma', country='USA', confidence=1)

### Serialize / Compress

In [25]:
import sys
from pprint import pprint
import zlib
# size of object in bytes
pprint(sys.getsizeof(add))
pprint(sys.getsizeof(add.json()))
pprint(sys.getsizeof(zlib.compress(add.json().encode())))

56
231
164
